# Extraction of socioeconomic variables

This notebook contains the extraction of county-level socioeconomic variables. This data is made available on the website www.regionalstatistik.de.

This website is based on the information system **genesis**. This information system is also used by other statistical websites such as www.bildungsmonitoring.de or https://www.statistikdaten.bayern.de/.

Genesis includes a SOAP/XML and a RESTful/JSON API for automatic data retrieval. For the programming language `R`, there exists a package with the name `wiesbaden` which contains functions for extracting data from this API. There is no comparable package for the programming language `Python`. 

To due this lack, we created an own module for extracting data fom a genesis information system. We studied the [genesis API documentaion](https://www-genesis.destatis.de/genesis/misc/GENESIS-Webservices_Einfuehrung.pdf) and named our module `wiesbaden`. This module contains a class named `Genesis`. This class has methods for searching and extracting the data. The functionality was leant on the R package `wiesbaden` (method names and output format).

## Preparatory activities

The first step is to import the packages/modules needed.

In [1]:
import pandas as pd
from pymongo import MongoClient

# our own package
import wiesbaden

Then we instantiate a new object of the class `Genesis` and call it `regiostat`. The `__init__` method sets the instance variables and checks if the database and userdata are correct. If not, a value error is raised.

In [2]:
regiostat = wiesbaden.Genesis(database = "REGIO", username = "username", password = "password")

> **Important note:**
>
> Both parameters (username, password) has been deleted. Please register at [Regionalstatistik](https://www.regionalstatistik.de/genesis/online/) to get access to the data.

## Relevant data cubes

Before we can query the data, it is important to know the name of the data cubes which contain the variables needed. There are two opportunities to do this.

The **first opportunity** is the method `search_datacube("Keyword")`. This method returns the code of datacubes whose contents contain a certain keyword in a pandas data frame. The following query returns all data cubes which contain the  gross domestic product (in german "BIP").

In [3]:
regiostat.search_datacube("BIP")

,Code,Content,State,Time,LatestUpdate,Information
0,82111BJ008,"VGR der Länder: Entstehungsrechnung, Bruttoinlandsprodukt, Bruttoinlandsprodukt je Erwerbstätigen, Bruttoinlandsprodukt je Einwohner, Deutschland, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:12:31h,false
1,82111KJ008,"VGR der Länder: Entstehungsrechnung, Bruttoinlandsprodukt, Bruttoinlandsprodukt je Erwerbstätigen, Bruttoinlandsprodukt je Einwohner, Kreise und kreisfreie Städte, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:12:52h,false
2,82111LJ008,"VGR der Länder: Entstehungsrechnung, Bruttoinlandsprodukt, Bruttoinlandsprodukt je Erwerbstätigen, Bruttoinlandsprodukt je Einwohner, Bundesländer, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:13:06h,false
3,82111RJ008,"VGR der Länder: Entstehungsrechnung, Bruttoinlandsprodukt, Bruttoinlandsprodukt je Erwerbstätigen, Bruttoinlandsprodukt je Einwohner, Regierungsbezirke / Statistische Regionen, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:13:21h,false
4,99910BJA17,"Regionalatlas Deutschland, BIP je Erwerbstätigen, Veränderung des BIP zum Vorjahr, BIP je EW, Deutschland, Jahr",vollständig mit Werten,2000-2019,22.12.2021 16:40:21h,false
5,99910BJC17,"Regionalatlas Deutschland, BIP je Arbeitsstunde, Deutschland, Jahr",vollständig mit Werten,2000-2017,31.01.2020 13:41:11h,false
6,99910KJA17,"Regionalatlas Deutschland, BIP je Erwerbstätigen, Veränderung des BIP zum Vorjahr, BIP je EW, Kreise und kreisfreie Städte, Jahr",vollständig mit Werten,2000-2019,22.12.2021 16:40:29h,false
7,99910KJC17,"Regionalatlas Deutschland, BIP je Arbeitsstunde, Kreise und kreisfreie Städte, Jahr",vollständig mit Werten,2000-2017,31.01.2020 13:41:13h,false
8,99910LJA17,"Regionalatlas Deutschland, BIP je Erwerbstätigen, Veränderung des BIP zum Vorjahr, BIP je EW, Bundesländer, Jahr",vollständig mit Werten,2000-2019,22.12.2021 16:40:31h,false
9,99910LJC17,"Regionalatlas Deutschland, BIP je Arbeitsstunde, Bundesländer, Jahr",vollständig mit Werten,2000-2017,31.01.2020 13:41:14h,false


The **second opportunity** is to go to the website http://www.regionalstatistik.de. On this website you need to find the **key** of the certain topic. Example: The keys of the **labour market** resources begin with the digits **13**.

![title](pictures/regiostat.PNG)

Then the obtained key can then be inserted in the method `retrieve_datalist("key")`. This method returns all data cubes whose codes contain a certain key.

In [4]:
regiostat.retrieve_datalist("13*")

,Code,Content,State,Time,LatestUpdate,Information
0,13111BJ001,"Statistik der sozialversicherungspflichtig Beschäftigten, Sozialversicherungspfl. Beschäftigte (Arbeitsort), Sozialversicherungspfl. Beschäftigte (Wohnort), Deutschland, Geschlecht, Stichtag",vollständig mit Werten,30.06.2008-30.06.2020,26.03.2021 15:15:15h,false
1,13111BJ002,"Statistik der sozialversicherungspflichtig Beschäftigten, Sozialversicherungspfl. Beschäftigte (Arbeitsort), Sozialversicherungspfl. Beschäftigte (Wohnort), Deutschland, Geschlecht, Nationalität, Stichtag",vollständig mit Werten,30.06.2008-30.06.2020,26.03.2021 15:15:16h,false
2,13111BJ003,"Statistik der sozialversicherungspflichtig Beschäftigten, Sozialversicherungspfl. Beschäftigte (Arbeitsort), Sozialversicherungspfl. Beschäftigte (Wohnort), Deutschland, Geschlecht, Beschäftigungsumfang, Stichtag",vollständig mit Werten,30.06.2008-30.06.2020,26.03.2021 15:15:18h,false
3,13111BJ004,"Statistik der sozialversicherungspflichtig Beschäftigten, Sozialversicherungspfl. Beschäftigte (Arbeitsort), Sozialversicherungspfl. Beschäftigte (Wohnort), Deutschland, Geschlecht, Nationalität, Beschäftigungsumfang, Stichtag",vollständig mit Werten,30.06.2008-30.06.2020,26.03.2021 15:15:19h,false
4,13111BJ005,"Statistik der sozialversicherungspflichtig Beschäftigten, Sozialversicherungspfl. Beschäftigte (Arbeitsort), Sozialversicherungspfl. Beschäftigte (Wohnort), Deutschland, Geschlecht, Altersgruppen (unter 20 bis 65 u. m.), Stichtag",vollständig mit Werten,30.06.2008-30.06.2020,26.03.2021 15:15:20h,false
...,...,...,...,...,...,...
135,13211RJ010,"Arbeitsmarktstatistik der Bundesagentur für Arbeit, Arbeitslosenquote bez. auf alle zivile Erwerbsp., Regierungsbezirke / Statistische Regionen, Arbeitslosenquote nach Schwerpunkten, Jahr",vollständig mit Werten,2009-2020,01.04.2021 10:04:09h,false
136,13312BJ001,"Erwerbstätigenrechnung des Bundes und der Länder, Erwerbstätige im Jahresdurchschn. (Inlandskonzept), Arbeitnehmer im Jahresdurchschn. (Inlandskonzept), Deutschland, WZ 2008-Abschnitte und Zusammenfassungen, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:39:08h,false
137,13312KJ001,"Erwerbstätigenrechnung des Bundes und der Länder, Erwerbstätige im Jahresdurchschn. (Inlandskonzept), Arbeitnehmer im Jahresdurchschn. (Inlandskonzept), Kreise und kreisfreie Städte, WZ 2008-Abschnitte und Zusammenfassungen, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:15:04h,false
138,13312LJ001,"Erwerbstätigenrechnung des Bundes und der Länder, Erwerbstätige im Jahresdurchschn. (Inlandskonzept), Arbeitnehmer im Jahresdurchschn. (Inlandskonzept), Bundesländer, WZ 2008-Abschnitte und Zusammenfassungen, Jahr",vollständig mit Werten,2000-2019,25.01.2022 11:15:19h,false


We searched our resources with the two methods described.

We needed to query data from the following data cubes:

* Gross Domestic Product: `82111KJ008`
* Unemployment Rate: `12411KJ019`
* Population: `12411KJ009`
* Size/Area: `11111KJ001`

## Query Data

This section contains the extraction and transfomation of all data. The first subsection (gross domestic product) contains a detailed description of the data extraction process and the methods. The other subsections contain only the most nessecary topics.

### Gross domestic product (GDP)

This subsection contains the extraction of the gross domestic product (data cube `82111KJ008`).

The data of a data cube can be retrieved with the method `retrieve_data()`.

In [4]:
# query data 
tmpGdp = regiostat.retrieve_data("82111KJ008")

# display top rows
tmpGdp.head(5)

,id82111,KREISE,JAHR,BIP802_val,BIP802_qual,BIP802_lock,BIP802_err,BIP803_val,BIP803_qual,BIP803_lock,BIP803_err,BIP804_val,BIP804_qual,BIP804_lock,BIP804_err
0,D,01001,2000,2487282,e,,0,43250,e,,0,30799,e,,0
1,D,01001,2001,2465393,e,,0,43390,e,,0,30630,e,,0
2,D,01001,2002,2635779,e,,0,47762,e,,0,32778,e,,0
3,D,01001,2003,2705233,e,,0,49998,e,,0,33590,e,,0
4,D,01001,2004,2867823,e,,0,53461,e,,0,35500,e,,0


The column names in the table are not informative. The description of the column names is retrieved with the method `retrieve_metadata()`.

In [5]:
regiostat.retrieve_metadata("82111KJ008")

,Code,Content,Type,Updated,Unit,Values,Timeslices
0,KREISE,Kreise und kreisfreie Städte,klassifizierend,07.01.2020 09:19:56h,NaN,NaN,NaN
1,JAHR,Jahr,zeitidentifizierend,24.04.2008 12:00:14h,NaN,NaN,NaN
2,BIP802,Bruttoinlandsprodukt,Ganzzahl,05.04.2017 13:52:02h,Tsd. EUR,,NaN
3,BIP803,Bruttoinlandsprodukt je Erwerbstätigen,Ganzzahl,05.04.2017 13:52:02h,EUR,,NaN
4,BIP804,Bruttoinlandsprodukt je Einwohner,Ganzzahl,05.04.2017 13:52:02h,EUR,,NaN


The following transformation steps have to be performed after the data extraction :
* The variables `BIP802` (gdp) and `BIP804` (gdp per capita) are only relevant. All columns with the suffix `_val` contain the needed values.
* We take values from 2018 because it is the most recent non-corona year with no missing values.
* The remaining columns need to be renamed to more meaningful names.

In [6]:
# only values for the year 2018
tmpGdp = tmpGdp.loc[tmpGdp.JAHR=="2018"]

# only countyId, GDP and GDP per capita
tmpGdp = tmpGdp[["KREISE","BIP802_val","BIP804_val"]]

# rename columns
tmpGdp = tmpGdp.rename(columns={"KREISE":"countyId",
                                "BIP802_val": "gdp", 
                                "BIP804_val":"gdpPerCapita"})
# display top rows
tmpGdp.head(2)

,countyId,gdp,gdpPerCapita
18,01001,3733513,41944
38,01002,11823188,47723


### Unemployment rate

This subsection contains the extraction of the unemployment rate (Data cube `82111KJ008`). We use the measure **unemployment rate as a percentage of all civilian employees** (column code `ERWP10`).

In [7]:
# query data 
tmpUnempl = regiostat.retrieve_data("13211KJ009")

# only values for the year 2019
tmpUnempl = tmpUnempl.loc[tmpUnempl.JAHR=="2019"]

# only countyId and unemployment rate as a percentage of all civilian employees
tmpUnempl = tmpUnempl[["KREISE","ERWP10_val"]]

# rename columns
tmpUnempl = tmpUnempl.rename(columns={"KREISE":"countyId",
                                      "ERWP10_val": "unemploymentRate"})

# display top rows
tmpUnempl.head(5)

,countyId,unemploymentRate
18,01001,8.0
38,01002,7.6
58,01003,7.3
78,01004,7.9
98,01051,5.5


### Population

This subsection contains the extraction of the population (data cube `82111KJ008`). The data cube contains the population seperated by gender (2 rows per county - one with the female population and one with the male population). The total population is sufficient for our project. Therefore an aggregation is performed. 

In [8]:
# query data 
tmpPop = regiostat.retrieve_data("12411KJ009")

# only values for the year 2019
tmpPop = tmpPop.loc[tmpPop.STAG=="31.12.2019"]

# aggregate (no differentiation between male and female)
tmpPop = tmpPop.groupby("KREISE", as_index=False).sum("BEVSTD_val")

# rename columns
tmpPop = tmpPop.rename(columns={"KREISE":"countyId",
                                "BEVSTD_val": "population"})

# display top rows
tmpPop.head(5)

,countyId,population
0,01001,90164
1,01002,246794
2,01003,216530
3,01004,80196
4,01051,133193


### Area size

This subsection contains the extraction of the area size (data cube `11111KJ001`). 

In [9]:
# query data 
tmpArea = regiostat.retrieve_data("11111KJ001")

# only values for the year 2019
tmpArea = tmpArea.loc[tmpArea.STAG=="31.12.2019"]

# only countyId and area size
tmpArea = tmpArea[["KREISE","FLC006_val"]]

# rename columns
tmpArea = tmpArea.rename(columns={"KREISE":"countyId",
                                  "FLC006_val": "areaSize"})

# display top rows
tmpArea.head(5)

,countyId,areaSize
24,01001,53.02
50,01002,118.65
76,01003,214.19
102,01004,71.66
128,01051,1428.17


### County names

In contrast to the previous subsections, the last subsection of the data extraction process contains the extraction of a nominal variable (variable **countyname**). The data of such variables can be queried with the method `retrieve_valuelabel()`.

In [10]:
# query data 
tmpCountyNames = regiostat.retrieve_valuelabel("KREISE")

# only countyId and area size
tmpCountyNames = tmpCountyNames[["Code","Content"]]

# rename columns
tmpCountyNames = tmpCountyNames.rename(columns={"Code":"countyId",
                                                "Content": "countyName"})

# display top rows
tmpCountyNames.head(5)

,countyId,countyName
0,01001,"Flensburg, kreisfreie Stadt"
1,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt"
2,01003,"Lübeck, Hansestadt, kreisfreie Stadt"
3,01004,"Neumünster, kreisfreie Stadt"
4,01051,"Dithmarschen, Landkreis"


## Join data

In this section, the queried sub data frames are merged together to one final data frame. Before merging, the number of rows in the sub-data frames is checked.

In [11]:
print("Number of rows:")
print("---------------")
print("Gdp",tmpGdp.shape[0])
print("Unemployment",tmpUnempl.shape[0])
print("Population",tmpPop.shape[0])
print("Area",tmpArea.shape[0])
print("County Names",tmpCountyNames.shape[0])

Number of rows:
---------------
Gdp 416
Unemployment 416
Population 416
Area 416
County Names 489


There are more county names (489) than rows in the data frames with measures (416). The reason for this is that there are county names of old counties that have been merged into larger counties and that exist no longer. Only counties that exist in the year of the measurement are available in the data frames with the measures. Old counties are included in the  data frame with the county names. We only need the counties that existed in the years 2018 and 2019. In this case, other counties can be excluded by using inner joins.

In [12]:
countyData = pd.merge(tmpCountyNames, tmpGdp, on="countyId", how="inner")
countyData = pd.merge(countyData, tmpUnempl, on="countyId", how="inner")
countyData = pd.merge(countyData, tmpPop, on="countyId", how="inner")
countyData = pd.merge(countyData, tmpArea, on="countyId", how="inner")

In [13]:
countyData

,countyId,countyName,gdp,gdpPerCapita,unemploymentRate,population,areaSize
0,01001,"Flensburg, kreisfreie Stadt",3733513,41944,8.0,90164,53.02
1,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",11823188,47723,7.6,246794,118.65
2,01003,"Lübeck, Hansestadt, kreisfreie Stadt",9367966,43219,7.3,216530,214.19
3,01004,"Neumünster, kreisfreie Stadt",3458069,43546,7.9,80196,71.66
4,01051,"Dithmarschen, Landkreis",4606985,34554,5.5,133193,1428.17
...,...,...,...,...,...,...,...
411,16073,"Saalfeld-Rudolstadt, Kreis",2839645,26573,5.3,103199,1008.79
412,16074,Saale-Holzland-Kreis,1929064,23236,4.1,82950,815.24
413,16075,Saale-Orla-Kreis,2253787,27761,4.6,80312,1151.30
414,16076,"Greiz, Kreis",2365566,23963,4.6,97398,845.98


## Adding State and Region

In this section, the `state` and `region` (East or West Germany) columns are added to the data frame. The first two digits of the `countyId` represent the id of a state. Therefore, this column can be used for mapping.

Initially, a data frame is created with the names of all states and regions.

In [14]:
# data list
stateListData = [['01', 'Schleswig-Holstein', 'West Germany'],
                ['02', 'Freie und Hansestadt Hamburg', 'West Germany'],
                ['03', 'Niedersachsen', 'West Germany'],
                ['04', 'Freie Hansestadt Bremen', 'West Germany'],
                ['05', 'Nordrhein-Westfalen', 'West Germany'],
                ['06', 'Hessen', 'West Germany'],
                ['07', 'Rheinland-Pfalz', 'West Germany'],
                ['08', 'Baden-Württemberg', 'West Germany'],
                ['09', 'Freistaat Bayern', 'West Germany'],
                ['10', 'Saarland', 'West Germany'],
                ['11', 'Berlin', 'East Germany'],
                ['12', 'Brandenburg', 'East Germany'],
                ['13', 'Mecklenburg-Vorpommern', 'East Germany'],
                ['14', 'Freistaat Sachsen', 'East Germany'],
                ['15', 'Sachsen-Anhalt', 'East Germany'],
                ['16', 'Freistaat Thüringen', 'East Germany']]


# list of column names
stateColumnNames = ["stateId", "state", "region"]

# create data frame
stateDf = pd.DataFrame(stateListData, columns=stateColumnNames)

# dispaly first two rows
stateDf.head(2)

,stateId,state,region
0,01,Schleswig-Holstein,West Germany
1,02,Freie und Hansestadt Hamburg,West Germany


Subsequently, the names of the states and regions are inserted into the main data frame by using an inner join.

In [15]:
# inner join
countyData = pd.merge(countyData, stateDf, left_on=countyData["countyId"].astype(str).str[0:2], right_on="stateId", how="inner")

# dispaly first two rows
countyData.head(2)

,countyId,countyName,gdp,gdpPerCapita,unemploymentRate,population,areaSize,stateId,state,region
0,01001,"Flensburg, kreisfreie Stadt",3733513,41944,8.0,90164,53.02,01,Schleswig-Holstein,West Germany
1,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",11823188,47723,7.6,246794,118.65,01,Schleswig-Holstein,West Germany


## Loading into Mongo DB

Finally, the data is loaded into the Mongo DB database.

The first step is to extract the data from the pandas data frame by using the method `to_dict()`. The result is a list with dictionaries as list elements. Each dictionary represents one county.

In [16]:
countyDataMongo = countyData.to_dict(orient="records")
countyDataMongo

[{'countyId': '01001',
  'countyName': 'Flensburg, kreisfreie Stadt',
  'gdp': 3733513,
  'gdpPerCapita': 41944,
  'unemploymentRate': 8.0,
  'population': 90164,
  'areaSize': 53.02,
  'stateId': '01',
  'state': 'Schleswig-Holstein',
  'region': 'West Germany'},
 {'countyId': '01002',
  'countyName': 'Kiel, Landeshauptstadt, kreisfreie Stadt',
  'gdp': 11823188,
  'gdpPerCapita': 47723,
  'unemploymentRate': 7.6,
  'population': 246794,
  'areaSize': 118.65,
  'stateId': '01',
  'state': 'Schleswig-Holstein',
  'region': 'West Germany'},
 {'countyId': '01003',
  'countyName': 'Lübeck, Hansestadt, kreisfreie Stadt',
  'gdp': 9367966,
  'gdpPerCapita': 43219,
  'unemploymentRate': 7.3,
  'population': 216530,
  'areaSize': 214.19,
  'stateId': '01',
  'state': 'Schleswig-Holstein',
  'region': 'West Germany'},
 {'countyId': '01004',
  'countyName': 'Neumünster, kreisfreie Stadt',
  'gdp': 3458069,
  'gdpPerCapita': 43546,
  'unemploymentRate': 7.9,
  'population': 80196,
  'areaSize': 

The next step is to establish a connection to the database server and to select the database `stepstone-data`.

In [17]:
# connect to database server
client = MongoClient('mongodb://localhost:27017/')

# select database
db = client["stepstone-data"]

The last step is to insert the data into the `counties` collection. Beforehand, existing data must be removed in order to avoid duplicates.

In [19]:
# delete old data
db.counties.delete_many({})

# insert new data
db.counties.insert_many(countyDataMongo)